## Imports

In [1]:
# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings
from intent_model import IntentModel

# Chat demo
import gradio as gr

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load data 

In [2]:
# Load the data from JSON file
with open("intent_recognition_v2.json", "r") as f:
    data = json.load(f)

validation_data = []
train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    validation = datum["test_questions"]
    for text in texts:
        train_data.append((text, label))
    for text in validation:
        validation_data.append((text, label))

# Define the mapping between top-level labels and integers
train_label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}
validation_label_map = {label: i for i, label in enumerate(set([data[1] for data in validation_data]))} 

# Convert the training data labels to integers using the label_map
train_labels = torch.tensor([train_label_map[data[1]] for data in train_data])
validation_labels = torch.tensor([validation_label_map[data[1]] for data in validation_data])
    

# Intent recognition model

In [3]:
intent_model = IntentModel(train_data, validation_data, train_label_map, train_labels, validation_label_map, validation_labels)

In [ ]:
intent_model = IntentModel(train_data, validation_data, train_label_map, train_labels, validation_label_map, validation_labels).test_best_model()

In [5]:
print(intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")[0], ": minimum age")
print(intent_model.get_intent("Could you summarize the main provisions of the legal statement?")[0], ": legal_statement_information")
print(intent_model.get_intent("Are you allowed to transfer ownership of my account or subscription to another individual?")[0], ": transfer_legal_statement")

use_restrictions_legal_statement : minimum age
privacy_policy_updates : legal_statement_information
personal_information : transfer_legal_statement


In [4]:
import pandas as pd

# test the intent model
predicted_intent, confidence_scores = intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")

# Retrieve the intent label and its corresponding confidence score
highest_confidence_intent = predicted_intent

print("Highest confidence intent:", highest_confidence_intent)
print("Highest confidence score:", confidence_scores)
print(len(confidence_scores))
print(len(intent_model.train_label_map.keys()))

pd.DataFrame({
    'keys': intent_model.train_label_map.keys(),
    'confidences': confidence_scores
})

Highest confidence intent: use_restrictions_legal_statement
Highest confidence score: [0.00521285692229867, 0.000778408139012754, 0.0007715903921052814, 0.005556902848184109, 0.0005540921702049673, 0.003890324616804719, 0.004483066499233246, 0.9502684473991394, 0.0013385297497734427, 0.0011972436914220452, 0.004930698312819004, 0.001752267125993967, 0.0021841523703187704, 0.0022397651337087154, 0.001835941686294973, 0.0017369056586176157, 0.0019594025798141956, 0.007882067933678627, 0.001427303534001112]
19
19


,keys,confidences
0,delete_personal_information,0.005213
1,data_storage,0.000778
2,data_collection,0.000772
3,contact_privacy_concerns,0.005557
4,data_breaches,0.000554
5,summarization_legal_statement,0.003890
6,privacy_policy_updates,0.004483
7,use_restrictions_legal_statement,0.950268
8,personal_information,0.001339
9,access_personal_data,0.001197


## ChatGPT model

In [ ]:
# GPT model here
gpt_model = GPT()

# gpt_model = False

# Test ChatGPT model
# gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


# Load Settings class

In [ ]:
settings_model = Settings()

# Test the settings model
# settings_model.update_user_setting("1", "profile_published", False)
settings_model.set_user_calling_card_visibility(user_id = "0", individual_user_id = "2", profile_card_component = "profile_card_component_3_visible", component_state = True)



## Get response using intent

In [ ]:
def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

# Set up gradio chat for demo

In [ ]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False,
             tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent, confidence_score = intent_model.get_intent(question=user_message)
        print('intent:', intent)
        print('confidence:', confidence_score)

        confidence_score = max(confidence_score)

        intent = get_object_by_intent(intent)


        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent['use_gpt']
        generate_context = True

        # if the intent is 'privacy_policy' or 'legal_statement', use GPT to generate a response
        response = ('(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message))
                    if use_gpt else
                    '<em><strong>(intent found ({}): {}):</strong></em> \n{}'.format(confidence_score, intent['intent'], intent['responses'][0]))
        # response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message)) if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])

        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        if not use_gpt:
            time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.launch()
